# Homework 7

In this homework, you will implement a multi-class Logistic Regression classifier and the SGD algorithm, and train a classifier to recognize images of digits in the [MNIST dataset](http://yann.lecun.com/exdb/mnist/). You will also investigate the significance of hyperparameter tuning for training and evaluation.

The purpose of this homework is to have you computing training objectives and gradients, and understand the role of regularization hyperparameter $\lambda$, learning rate $\eta$, batch size, and batch sampling techniques for training with SGD.

There are a number of programming **tasks** and **quiz questions** in this homework.
- For **tasks**, you will need to either **add code between comments "`#### TASK N CODE`"** to complete them or **modify code between those comments**. **DO NOT delete the comments "#### TASK N CODE". This is for graders' reference and you might not get full points if you tamper with these comments.**
- For **quiz questions**, you will need to answer in a few sentences between the given lines.
- For **optional tasks**, you are **NOT required to turn them in**. However, we encourage you to complete them as they are good practice.
- For **challenge-optional tasks**, you are **NOT required to turn them in**. However, you will receive extra credit for completing the challenge.

---

First, we will install the `python-mnist` [library](https://github.com/sorki/python-mnist) that provides helper functions to convert the MNIST dataset into numpy arrays.

In [ ]:
!gdown 1Az1TQNayBnQJ9ofcByGFP0BPbo2BcKvb
!wget -O $PWD/linclass.py https://www.dropbox.com/scl/fi/lg1d7w15dyrave6c5nw1c/linclass.py?rlkey=crysv0fe00pq5ao2hhx9puk0x&dl=1

In [ ]:
!pip install python-mnist
!pip install scikit-image

Modules `linclass` and `utils` contain several helper functions from previous homeworks such as `empirical_err()` and `create_split()`, which you might need for this homework. We encourage you to read through the relevant functions.

In [ ]:
from dataclasses import dataclass
from typing import Any, Callable, Dict, List, Optional, Tuple

import linclass
import matplotlib.pyplot as plt
import numpy as np
import utils

In [ ]:
SEED = 0
np.random.seed(SEED)

### Read MNIST Data

MNIST is a large dataset of images of handwritten digits, which can be used to train a classifier that predicts the digit (0 through 9) given a black and white image of the digit. `utils.read_MNIST` is a helper function that downloads the MNIST data to a local directory `mnist_data`, creates training, validation, and testing splits, and loads the data as numpy arrays.

MNIST has 60000 training examples and 10000 testing examples. We keep 20% of the training examples for validation, giving you 48000 training examples ultimately.

Each image is a 28 x 28 size array of ones and zeros, denoting black and white colors respectively. Let's first load the data and visualize a few images.

In [ ]:
xTr, yTr = utils.read_MNIST('training')
xVal, yVal = utils.read_MNIST('validation')
xTe = utils.read_MNIST('testing')

In [ ]:
print(f'xTr: {xTr.shape}, yTr: {yTr.shape}')
print(f'xVal: {xVal.shape}, yVal: {yVal.shape}')
print(f'xTe: {xTe.shape}')

In [ ]:
print('Training images')

print(f'Class: {yTr[10]}')
utils.show_MNIST_example(xTr[10])

print(f'Class: {yTr[20]}')
utils.show_MNIST_example(xTr[20])

print('Validation images')

print(f'Class: {yVal[10]}')
utils.show_MNIST_example(xVal[10])

print(f'Class: {yVal[20]}')
utils.show_MNIST_example(xVal[20])

print('Test images')

print(f'10th test example')
utils.show_MNIST_example(xTe[10])

print(f'20th test example')
utils.show_MNIST_example(xTe[20])

### Working with Truncated Training Set

In order to ensure that the entire notebook runs in the allowed 12 GB RAM budget of a colab notebook, we will be working with the first 20,000 example of the trainin set.

However, for the Kaggle competition, it is really advised to work with the full training examples available to you (potentially in a different notebook), and submit the best possible results.

In [ ]:
xTr, yTr= xTr[:20000], yTr[:20000]

### Feature normalization

Because gradient descent is sensitive to feature scale, we need to standardize the features (pixels of images) so that each feature has mean 0 and standard deviation 1 across the dataset. We will use the training set to estimate the mean and standard deviation of the digits, and standardize images $x$ as part of producing features $\phi(x)$.

`per_coordinate_normalized_phi` outputs a feature map that does the following transformations on an image $x \in \mathbb{R}^{d \times d}$, i.e. of width and height $d$:
1. Flatten $x$ to get a $d^2$-dimensional vector.
2. Standardize the vector representation with estimates of mean and standard deviation from the training set.
3. Apply the `base_phi` feature map. The default is `affine_phi` which adds a 1 to the beginning of the vector representation for the image for the bias parameter.

In [ ]:
def per_coordinate_normalized_phi(
    xTr: np.ndarray,
    base_phi = linclass.affine_phi,
) -> Callable[[np.ndarray], np.ndarray]:
    '''
    Returns a phi mapping that normalizes the scale of each feature using
    training data X and applies base_phi.

    Args:
        X: Data features. shape (n, ...)
        base_phi: (default linclass.affine_phi) Feature mapping to apply after normalization.

    Returns:
        phi: Function from dataset of shape (m, d') to phi mappings (m, d')
    '''
    n = xTr.shape[0]
    xTr = xTr.reshape(n, -1) # Make vectors for each data point

    # Compute normalizing stats
    mu = xTr.mean(axis=0, keepdims=True)
    sigma = xTr.std(axis=0, keepdims=True) + 1e-6

    def phi(X: np.ndarray) -> np.ndarray:
        X = X.reshape(X.shape[0], -1) # Make vectors for each data point
        X_normalized = (X - mu) / sigma
        return base_phi(X_normalized)
    return phi

We can visualize the results of applying the `per_coordinate_normalized_phi` to training images.

In [ ]:
print('Before normalization')
utils.show_MNIST_example(xTr[30])
utils.show_MNIST_example(xTr[70])

norm_affine_phi = per_coordinate_normalized_phi(xTr, base_phi=linclass.affine_phi)
X = norm_affine_phi(xTr)
print('After normalization')
utils.show_MNIST_example(X[30])
utils.show_MNIST_example(X[70])

## Building a Multiclass Classifier

Now we turn to designing a multiclass classifier. Given a feature mapping $\phi : \mathcal{X} \rightarrow \mathbb{R}^d$ and data comprising of image-label pairs $(x, y) \in \mathcal{X} \times \mathcal{Y}$ where $\mathcal{Y} = \{0, \dots, 9\}$, we want to obtain a logistic regression classifier $h_w : \mathcal{X} \rightarrow \mathcal{Y} $ with parameters $w$. We will use Structural Risk Minimization (Empirical Risk Minimization with Regularization) to build a logistic regression classifier. That is,
$$
\hat{w} = \arg\min_{w} \frac{1}{m} \sum_{i=1}^m \ell^{lgst} \left( h_w \left( x_i \right), y_i \right) + \lambda R(w) \qquad \text{where } R \text{ is a regularizer}.
$$

As you have seen previously, the multiclass logistic loss (also called Cross Entropy) uses linear predictors $r_y (x) = \left\langle w_y, \phi(x) \right\rangle$ for each label $y$. The conditional probability of label $y$ given $x$ uses softmax:
$$
P (y \mid x; w) = \frac{\exp \left( r_y(x) \right)}{\sum_{y' \in \mathcal{Y}} \exp \left( r_{y'}(x)\right) }
$$
and thus the multiclass logistic loss is:
$$
\ell^{lgst} \left( h_w(x), y \right) = - \log P (y \mid x; w) = - \log \frac{\exp \left( r_y(x) \right)}{\sum_{y' \in \mathcal{Y}} \exp \left( r_{y'}(x)\right) }
$$

As we have seen before, minimizing the logistic class is equivalent to maximizing the conditional likelihood of the training set. The parameters of $h_w$ are $w = \left[ w_{y_1}, \dots, w_{y_k}\right] \in \mathbb{R}^{d \times k}$ where $k$ is the number of labels ($|\mathcal{Y}| = k$).

Having defined the multiclass linear model and the loss, we can optimize the objective using our choice of a regularizer on the parameters, such as the L2 and the L1 regularizers. These are $||w||_F^2 = \sum_{y \in \mathcal{Y}} ||w_y||_2^2$ and $\sum_{y \in \mathcal{Y}} ||w_y||_1$ respectively.

You will use Minibatch Stochastic Gradient Descent to optimize the training objective. Therefore, you will calculate the gradient of the objective w.r.t. parameters $w$ and perform gradient steps to obtain the Structural Risk Minimization solution.

### Numerical Stability

You might have noticed overflow errors in using the logistic loss in the last homework. Before proceeding to implementing the multi-class logistic loss, let us understand and prevent these errors. Our simplistic implementation of the logistic loss was:

In [ ]:
def logistic_loss(z):
    return np.log(1. + np.exp(-z))

This is mathematically correct, and works fine as long as $z$ isn't too negative.  But once $z$ is very negative, $\exp(-z)$ becomes too large and overflows:

In [ ]:
for z in [10000, 1000, 100, 10, 5, 1, -1, -5, -10, -100]:
    print(z, logistic_loss(z))

In [ ]:
logistic_loss(-1000)

In the last programming homework, you might have noticed that we suppressed some numpy warnings. This was to avoid divide-by-zero errors popping up when `np.log` encounters values very close to 0 and overflow errors when `np.exp` encounters value very negative (though you might have seen warnings even after we had suppressed a majority).

Thus, the problems are when:
- Computing $\log \frac{\exp y}{\exp x}$ for $x \gg y$ in fixed-precision floating point math. Although with unlimited precision the operation should exactly equal $y - x$, taking the exponent of a large number and using it as a denominator in fixed precision often rounds the fraction to 0, on which `np.log` complains.
- Computing $\exp(-z)$ when $z$ is very negative. This gets rounded off to $\infty$.

When expressions in floating-point arithmetic round to 0 due to fixed precision, they throw **underflow** errors. Likewise, when they round to $\infty$ due to fixed precision, they throw **overflow** errors.

In the last code cell, even though the intermediate calculation $\exp(1000)$ is too large, the final output after applying the log is a very sensible number, namely $-1000$.  But our implementation misses this, as it gets lost due to the exponentiation overflow. To fix this, we can implement `logistic_loss` more carefully, avoiding taking exponents of large numbers.  We do so by simplifying the logistic loss as:

$$
\log(1+e^{-z})=\log(1+e^{-z})=\log(e^{\max(0,-z)}(e^{-\max(0,-z)}+e^{-z-\max(0,-z)})=\max(0,-z)+\log(e^{\min(0,z)}+e^{\min(0,-z)}).
$$

This expression is equivalent mathematically, but note that we are only exponentiating negative numbers, and so will never have an overflow in the exponentiation.

In [ ]:
def safe_logistic_loss(z):
    return np.maximum(0, -z) + np.log(np.exp(np.minimum(0, z)) + np.exp(np.minimum(0, -z)))

We can verify that `safe_logistic_loss` returns the same answers as logistic when $z$ isn't too negative (and might also a bit more accurate and numerically stable when $z$ is very negative):

In [ ]:
for z in [10000, 1000, 100, 10, 5, 1, -1, -5, -10, -100, -500, -700, -1000]:
    print(z, logistic_loss(z), safe_logistic_loss(z))

### Numerically Stable Softmax

We will now program the softmax function:
$$
\text{softmax}(r) = \frac{\exp(r_i)}{\sum_j \exp(r_j)}.
$$

Naively, one would code `softmax(r)` on a vector `r` as the following:

```python
def softmax(r):
    # Numerically unstable!
    num = np.exp(r)
    return num / np.sum(num)
```

If one of the values in `r` was large relative to the others, `softmax` would return 0 for the others due to underflow errors, and applying `log` (to calculate the logistic loss for example) will be problematic. Therefore, we need to make `softmax` numerically stable. We do so by dividing the numerator and denominator by the largest entry in the exponentiated vector. That is,
$$
\text{softmax}(r) = \frac{\exp(r_i)}{\sum_j \exp(r_j)} = \frac{\exp(r_i) / \exp(r_{max})}{\sum_j \exp(r_j) / \exp( r_{max})} = \frac{\exp(r_i - r_{max})}{\sum_j \exp(r_j - r_{max})}.
$$

In [ ]:
def softmax(r: np.ndarray) -> np.ndarray:
    '''
    Applies the softmax operation on each row of the input, thus converting
    to normalized probabilities.

    Args:
        r: Inputs to normalize. shape (n, k)

    Returns:
        p: shape (n, k), where p[t, i] = exp(r[t, i]) / sum_j exp(r[t, j]).
    '''
    stable_num = np.exp(r - np.max(r, axis=1)[:, np.newaxis])
    return stable_num / np.sum(stable_num, axis=1)[:, np.newaxis]

### Definitions for the Classifier

Now we can proceed with defining the class structure for the classifiers. We will first define a `MultiLinearClassifier` derived from `Classifier` in the `linclass` module. The `MultiLinearClassifier` can be instantiated with a `phi` function, that applies a transformation to raw data features `X` to output a $d$-dimensional vectors for each of the inputs. The default is the affine transformation ($x \mapsto (1, x')$ where $x'$ is the vector representation of $x$).

The classifier uses the parameters $w \in \mathbb{R}^{d \times k}$ in linear prediction and picks the labels corresponding to the highest responses (we need not apply a softmax operation as it is just converts responses to probabilities without changing the relative values among predictions for different classes).

In [ ]:
class MultiLinearClassifier(linclass.Classifier):
    '''
    Multiclass predictor with k classes based on linear predictors
    in a matrix self.w, where self.w[:, j] is the predictor for class j.
    '''
    def __init__(self, phi: Callable[[np.ndarray], np.ndarray] = linclass.affine_phi):
        '''
        Args:
            phi: (default linclass.affine_phi) Function returning a feature mapping for input features.
        '''
        self.phi = phi

    def predict(self, X: np.ndarray) -> np.ndarray:
        '''
        Returns predicted labels for data.

        Args:
            X: Raw data features (self.phi applied before using linear prediction).
                shape (m, d')

        Returns:
            shape (m)
        '''
        return np.argmax(self.phi(X) @ self.w, axis=1)

### [Task 1] Complete the SGD training function

You will first complete the SGD implementation. `SGD` accepts the initial parameters `w0`, a gradient calculator `grad_calculator` (function from `(w, batch_idxs)` to gradient of objective w.r.t. `w`), and training parameters such as batch size, learning rate, data sampling techniques etc. We have redacted the gradient step in the heart of the implementation for you to write.

In [ ]:
@dataclass
class SGDLogger:
    '''
    Class for recording logs.

    Usage:
        ```
        # log attribute can either be given during instantiation
        # or will be populated during SGD using logging_func(w) outputs.
        SGDLogger(
            name='l2_norm_weights',
            logging_func=lambda w: np.linalg.norm(w, 'fro'),
            log=None,
            can_display=True,
            per_epoch=False
        )
        ```

    Args:
        name: Name for the logger.
        logging_func: Function from weights to some value that will be logged.
        log: (default None) Logged values.
        can_display: (default True) Flag for whether log can be printed neatly.
        per_epoch: (default False) Flag for whether logging_func(w) should
            be called per epoch in SGD method.
    '''
    name: str
    logging_func: Callable[[np.ndarray], Any] # f : w -> any
    log: Any = None # Initialize to None
    can_display: bool = True # Flag if log is displayable
    per_epoch: bool = False # Flag if logging_func should be called per epoch


def SGD(
    w0: np.ndarray,
    grad_calculator: Callable[[np.ndarray, Optional[np.ndarray]], np.ndarray],
    m: int,
    batch_size: int = 32,
    eta: float = 0.01,
    n_epochs: int = 10,
    sampling: str = 'epoch_shuffle',
    loggers: List[SGDLogger] = [],
    verbose: bool = True
) -> np.ndarray:
    '''
    Optimizes the parameters initialized at w using MiniBatch SGD on the dataset of size m
    and returns the final parameters of the classifier.

    Args:
        w0: Initial parameters for SGD. Any shape.
        grad_calculator: Function with (w, idxs) as inputs where w are parameters
            the same shape as w0 and idxs is an optional array of samples' indices,
            returning an estimate of the gradient at w based on samples with
            those indices.
        m: Size of training set.
        batch_size: (default 32) Size for mini batch.
        eta: (default 0.1) Learning rate of the MiniBatch SGD algorithm.
        n_epochs: (default 10) Number of epochs to train for.
        sampling: (default 'epoch_shuffle') one of: ['cyclic', 'randperm', 'epoch_suffle', 'iid'].
            'cycling': cycle over data in input order.
            'randperm': cycle over a random permutation of data fixed across epochs.
            'epoch_shuffle': cycle over a random permutation of data shuffled randomly every epoch.
            'iid': iid sample from the m points every epoch.
        loggers: (default []) List of SGDLoggers, the logging functions of
            which will be called during training (frequency determined by per_epoch).
        verbose: (default True) Flag to display information while training.

    Returns:
        w: shape (d, num_labels) model parameters
    '''
    assert sampling in ['cyclic', 'randperm', 'epoch_shuffle', 'iid'], 'Unknown sampling method'

    w = w0

    for logger in loggers:
        if logger.per_epoch:
            logger.log = []

    if sampling == 'randperm':
        # One random permutation for all epochs
        shuffle_idxs = np.random.permutation(m)
    elif sampling == 'cyclic':
        # Cycle over data in input order
        shuffle_idxs = np.arange(m)

    for epoch in range(n_epochs):
        if sampling == 'epoch_shuffle':
            # Sample without replacements each epoch,
            # i.e. use an independently sampled permutation each round
            shuffle_idxs = np.random.permutation(m)
        elif sampling == 'iid':
            # iid sampling, as in SGD theory
            shuffle_idxs = np.random.randint(0, high=m, size=m)
        n_batches = m // batch_size
        batch_idxs = np.array_split(shuffle_idxs, n_batches)

        # Train on mini batch
        for b in range(n_batches):
            b_idxs = batch_idxs[b] # the samples to use in this minibatch

            #### TASK 1 CODE
              # the stochastic gradient estimate
              # gradient step
            #### TASK 1 CODE

        # Log per epoch loggers
        for logger in loggers:
            if logger.per_epoch:
                logger.log.append(logger.logging_func(w))
        if verbose:
            if epoch == 0:
                print()
            s = [f'--- Epoch: {epoch}']
            for logger in loggers:
                if logger.can_display and logger.per_epoch:
                    s.append(f'{logger.name}: {logger.log[-1]:5}')
            if len(s) > 1:
                print(', '.join(s))

    # Log final loggers
    for logger in loggers:
        if not logger.per_epoch:
            logger.log(logger.logging_func(w))
    if verbose:
        s = [f'Training complete']
        for logger in loggers:
            if logger.can_display and (not logger.per_epoch):
                s.append(f'{logger.name}: {logger.log:5}')
        if len(s) > 1:
            print(', '.join(s))

    return w

### Fitting the MultiLinearClassifier using ERM

Next we define the `ERMMultiLinearClassifier` classifier that trains the `MultiLinearClassifier` using SGD. We preserve the structural similarity with `linclass.ERMLinearClassifier` by defining `train_obj` and `train_grad` as closures that accept weights; however, we use SGD for training rather than `scipy.optimize.minimize`. `train_obj` uses a function `loss_func` that computes the loss of multiple data points, and `train_grad` uses `loss_grad` that computes the gradient w.r.t. responses $r(X) = \phi(X) w \in \mathbb{R}^{m \times k}$ ($m$ data points).

In [ ]:
class ERMMultiLinearClassifier(MultiLinearClassifier, linclass.ERMLinearClassifier):
    '''
    Multiclass predictor trained by minimizing the (regularized) empirical risk with SGD,
    w.r.t. some loss function.
    '''
    def __init__(self, lmb: float = 0., **kwargs):
        '''
        Args:
            lmb: (default 0) Regularization tradeoff prameter.
        '''
        super().__init__(**kwargs)
        self.lmb = lmb

    def fit(self, X: np.ndarray, y: np.ndarray, **sgd_kwargs):
        '''
        Fits the classifier on dataset, with self.phi applied on the data features.

        Args:
            X: Raw data features (self.phi applied before using linear prediction).
                shape (m, d')
            y: Data labels, 0 <= y_i < k. shape (m)
        '''
        phiX = self.phi(X)
        m, d = phiX.shape

        # Define training objective
        def train_obj(w: np.ndarray, batch: Optional[np.ndarray] = None) -> float:
            '''
            Calculates the training objective with parameters w on a batch of training samples.

            Args:
                w: shape (d, k), where d is the dimension of self.phi(X) and k are the number
                    of labels.
                batch: (default None) Indices of samples to calculate objective on. If None,
                    calculate objective on all samples.
            '''
            if batch is None:
                # All data is in a batch
                batch = slice(None)

            phiXbatch = phiX[batch]
            loss = np.mean(self.loss_func(phiX[batch] @ w, y[batch]))
            reg = self.regularizer(w)
            return loss + self.lmb*reg

        # Define training gradient
        def train_grad(w: np.ndarray, batch: Optional[np.ndarray] = None) -> np.ndarray:
            '''
            Returns the gradient of the training objective w.r.t. parameters w,
            calculated on a batch of training samples.

            Args:
                w: shape (d, k), where d is the dimension of self.phi(X) and k are the number
                    of labels.
                batch: (default None) Indices of samples to calculate objective on. If None,
                    calculate objective on all samples.
            '''
            if batch is None:
                # All data is in a batch
                batch = slice(None)

            phiXbatch = phiX[batch]
            b = phiXbatch.shape[0]
            loss_g = phiXbatch.T @ self.loss_grad(phiXbatch @ w, y[batch]) / b
            reg_g = self.regularizer_grad(w)
            return loss_g + self.lmb*reg_g

        k = np.max(y)+1
        w0 = np.zeros((d, k))

        self.sgd_loggers = [
            SGDLogger('train_obj', train_obj, can_display=True, per_epoch=True),
        ] + sgd_kwargs.pop('loggers', [])

        # Optimize using SGD
        self.w = SGD(
            w0,
            train_grad,
            m,
            loggers=self.sgd_loggers,
            **sgd_kwargs
        )

### [Task 2] Compute Logistic Loss and Gradient

In this task, you will complete the functions `loss_func` and `loss_grad`. Recall the logistic loss on a data point $(x_i, y_i)$:
$$
\ell^{lgst}(h_w(x_i), y_i) = - \log P(y_i \mid x_i; w) = - \log \frac{\exp \left( r_{y_i} (x_i) \right)}{\sum_{y' \in \mathcal{Y}} \exp \left( r_{y'}(x_i)\right) }
$$

- In `loss_func`, compute the logistic loss for a bunch of data points. One way would be to use `softmax` and take the log of the relevant probabilities, though `np.log` might complain (why?). The more stable way would be to simplify the expression of the logistic loss a little to obtain a more numerically stable implementation. **You should code the stable implementation of the loss.**

- In `loss_grad`, compute the gradient of the logistic los w.r.t. responses. First, write the expression for the gradient.

<span style="color: red">
<h4 style="font-weight: bold">[Answer Question 1]</h4>

What is the gradient of $\ell^{lgst}$ w.r.t. responses $r(x)$ for one data point $x$ and its label $y$?

<h4 style="font-weight: bold">---------------------</h4>

<span style="color: blue">
Answer:


</span>

<h4 style="font-weight: bold">---------------------</h4>
</span>

In [ ]:
class MultiLogisticReg(ERMMultiLinearClassifier):
    '''
    Multiclass predictor using Logistic Loss.
    '''
    def __init__(self, lmb: float = 0., **kwargs):
        super().__init__(lmb, **kwargs)

    def loss_func(self, responses: np.ndarray, y: np.ndarray) -> np.ndarray:
        '''
        Returns the logistic loss for each point calculated on predictor's responses against labels.

        Args:
            responses: Unnormalized responses of the predictor for each point for each label.
                shape (m, k)
            y: True labels, 0 <= y_i < k. shape (m)

        Returns:
            loss: shape (m), the logistic loss. loss[i] = -log(softmax(responses)[i, y[i]])
        '''
        #### TASK 2 CODE



        #### TASK 2 CODE

        return loss

    def loss_grad(self, responses: np.ndarray, y: np.ndarray) -> np.ndarray:
        '''
        Returns the gradient of logistic loss w.r.t. responses.

        Args:
            responses: Unnormalized responses of the predictor for each point for each label.
                shape (m, k)
            y: True labels, 0 <= y_i < k. shape (m)

        Returns:
            grad: shape (m, k) where grad[t, i] is the gradient w.r.t. responses[t, i].
            grad[t, i] = grad( -log((y[t] == i)*exp(responses[t, i]) / sum_j exp(responses[t, j])) )
                = grad( -( (y[t] == i)*responses[t, i] - log(sum_j exp(responses[t, j]))) )
                = -((y[t] == i) - exp(responses[t, i]) / sum_j exp(responses[t, j]))
        '''
        m, k = responses.shape

        #### TASK 2 CODE



        #### TASK 2 CODE

        return grad

### [Task 3] Adding Regularization

Finally, we need to add regularization and its gradient. The `ERMMultiLinearClassifier.fit` method calls the `regularizer` and `regularizer_grad` functions, that you will implement. Complete the functions for both L2 and L1 regularization.

**Important:** If `is_affine_phi` is `True`, the `affine_phi` was applied to raw data features and so a constant dimension was added for the bias parameter. You should NOT include the bias parameters, which are in the first row of the parameter matrix `w`, when calculating the regularization penalty. Consequently, the gradient of the regularizer w.r.t. bias parameters should also be set to 0 if `is_affine_phi` is `True`.

In [ ]:
class MultiLogL2Reg(MultiLogisticReg):
    def __init__(self, lmb: float = 0., is_affine_phi: bool = True, **kwargs):
        '''
        Args:
            is_affine_phi: (default True) Flag determining whether argument phi
                is an affine phi (adding constant dimension to features).
                This affects the regularizer calculation as the bias parameters
                are not penalized.
        '''
        super().__init__(lmb, **kwargs)
        self.is_affine_phi = is_affine_phi

    def regularizer(self, w: np.ndarray) -> float:
        #### TASK 3 CODE
        # All rows except the first, since the first row have the bias parameters
        reg =

        #### TASK 3 CODE

        return reg

    def regularizer_grad(self, w: np.ndarray) -> np.ndarray:
        #### TASK 3 CODE
        reg_grad =

        #### TASK 3 CODE

        return reg_grad


class MultiLogL1Reg(MultiLogisticReg):
    def __init__(self, lmb: float = 0., is_affine_phi: bool = True, **kwargs):
        '''
        Args:
            is_affine_phi: (default True) Flag determining whether argument phi
                is an affine phi (adding constant dimension to features).
                This affects the regularizer calculation as the bias parameters
                are not penalized.
        '''
        super().__init__(lmb, **kwargs)
        self.is_affine_phi = is_affine_phi

    def regularizer(self, w: np.ndarray) -> float:
        #### TASK 3 CODE
        # All rows except the first, since the first row have the bias parameters
        reg =
        reg =
        #### TASK 3 CODE
        return reg

    def regularizer_grad(self, w: np.ndarray) -> np.ndarray:
        #### TASK 3 CODE
        reg_grad =
        #### TASK 3 CODE
        return reg_grad

## Training the Classifier

Having defined all relevant functions, we can now train and use the classifier. First, we will load the training and validation data and obtain `norm_affine_phi` that standardizes the raw features and applies an affine transformation (constant dimension to the feature vector).

In [ ]:
norm_affine_phi = per_coordinate_normalized_phi(xTr, base_phi=linclass.affine_phi)

### [Task 4] Finding the best regularization hyperparameter

We will train the Multiclass logistic classifier with L2 regularization (class `MultiLogL2Reg`) on a range of regularization hyperparameters $\lambda$.  We have defined the `tune_regularization` function and called it in the next code cell for you, so that you can see how to use the classes above.

In this task, you will find the best $\lambda$ for the logistic classifier (recall: the goal is to get a classifier that generalizes well). Also plot the training and validation errors vs $\lambda$.
- We experiment with $\lambda$ in a logarithmic range, so please use logscale on the relevant axis.
- Make sure you label your axis and provide legends in the plot!

In [ ]:
def tune_regularization(
    xTr: np.ndarray, yTr: np.ndarray, xVal: np.ndarray, yVal: np.ndarray,
    multi_logi_classifier: MultiLogisticReg,
    lmbs: List[float],
    classifier_kwargs: Dict[str, Any] = {},
    sgd_kwargs: Dict[str, Any] = {},
    verbose: bool = True,
) -> List[MultiLogisticReg]:
    '''
    Returns trained classifiers, instantiated with classifier_kwargs
    and trained with sgd_kwargs. Each trained classifier to the
    corresponding regularization hyperparameter lmb in lmbs.

    Args:
        xTr: Training raw features. shape (mTr, *)
        yTr: Training labels. shape (mTr)
        xVal: Validation raw features. shape (mVal, *)
        yVal: Validation labels. shape (mVal)
        lmbs: A list of regularization hyperparameter values (lambdas).
        multi_logi_classifier: Class name of the multiclass logistic classifier.
        classifier_kwargs: (default {}) Keyword arguments to instantiate
            multiclass logistic classifier, supplied as a dictionary.
        sgd_kwargs: (default {}) Keyword arguments for SGD training,
            supplied as a dictionary.
        verbose: (default True) Flag for printing progress.

    Returns:
        clfs: List of trained classifiers. clfs[i] classifier is trained
            with lmb=lmbs[i].
    '''
    sgd_kwargs['verbose'] = sgd_kwargs.pop('verbose', True) and verbose
    clfs = []
    for lmb in lmbs:
        if verbose:
            print(f'lambda: {lmb:.2e} ::: ', end='')

        # Train classifier
        clf = multi_logi_classifier(lmb=lmb, **classifier_kwargs)
        clf.fit(xTr, yTr, **sgd_kwargs)

        # Log train and val err
        yTr_pred = clf.predict(xTr)
        train_err = utils.empirical_err(yTr, yTr_pred)
        train_err_logger = SGDLogger(
            'train_err', None, train_err, can_display=True, per_epoch=False
        )
        clf.sgd_loggers.append(train_err_logger)

        yVal_pred = clf.predict(xVal)
        val_err = utils.empirical_err(yVal, yVal_pred)
        val_err_logger = SGDLogger(
            'val_err', None, val_err, can_display=True, per_epoch=False
        )
        clf.sgd_loggers.append(val_err_logger)

        if verbose:
            print(f'train_err: {train_err:5f}, val_err: {val_err:5f}')

        clfs.append(clf)

    return clfs

Below is the code for running the tuning experiment. Select the best model/parameters based on the hyperparameter tuning, and plot errors.

In [ ]:
# Values below are a recommendation for initial exploration -- you should see if you want to refine these
lmbs = np.sort(np.concatenate([
    np.logspace(-5, -1, 5),
    np.array([0., 1., 5.]),
]))

classifier_kwargs = {
    'phi': norm_affine_phi,
    'is_affine_phi': True,
}

# Keyword arguments for the SGD method wrapped as a dictionary, opened when calling tune_regularization
# Feel free to add to this.
sgd_kwargs = {
    'batch_size': 16,
    'n_epochs': 20,
    'eta': 0.1,
    'verbose': False, # Disable printing INSIDE SGD
}

clfs = tune_regularization(
    xTr, yTr, xVal, yVal,
    MultiLogL2Reg,
    lmbs,
    classifier_kwargs=classifier_kwargs,
    sgd_kwargs=sgd_kwargs,
    verbose=True
)

# Select best classifier
## Get logged validation errors for the trained classifiers
val_errs = [logger.log
            for clf in clfs
            for logger in clf.sgd_loggers
            if logger.name == 'val_err']
#### TASK 4 CODE
best_idx =
best_lmb =
best_clf =
#### TASK 4 CODE

print(best_lmb)

# Plot training and validation error w.r.t. lambda
#### TASK 4 CODE

#### TASK 4 CODE
plt.figure()
plt.loglog(lmbs, train_errs, marker='.', label='train err')
plt.loglog(lmbs, val_errs, marker='.', label='val err')
plt.ylabel('empirical err')
plt.xlabel('$\\lambda$')
plt.legend()
plt.show()

### [Task 5] Finding the best regularization hyperparameter for L1

In this task, you will use `tune_regularization` to find the best $\lambda$ for L1-regularized logistic classifier. As above, also plot the training and validation error vs $\lambda$.

In [ ]:
#### TASK 5 CODE

clfs = tune_regularization(
    xTr, yTr, xVal, yVal,
    MultiLogL1Reg,
    lmbs,
    classifier_kwargs=classifier_kwargs,
    sgd_kwargs=sgd_kwargs,
    verbose=True
)

# Select best classifier
## Get logged validation errors for the trained classifiers
val_errs =

best_idx =
best_lmb =
best_clf =

print(best_lmb)

# Plot training and validation error w.r.t. lambda
train_errs =
plt.figure()



plt.ylabel('empirical err')
plt.xlabel('$\\lambda$')
plt.legend()
plt.show()
#### TASK 5 CODE

### [Task 6] Feature Normalization

Previously, we mentioned that applying normalization to features helps improve the performance of SGD optimization.
In this task, we would like to verify this idea by comparing the performance of classifiers trained with and without feature normalization.

Since feature normalization changes the magnitudes of the input data and affects the magnitudes of optimal model weights accordingly, we implement the "tune_learning_rate" method to find the best learning rate and ensure that the models are fully optimized. We also turn off regularization to simplify the experiment.

Please finish the missing code below and report the validation performance of classifiers trained with and without feature normalization.

In [ ]:
def tune_learning_rate(
    xTr: np.ndarray, yTr: np.ndarray, xVal: np.ndarray, yVal: np.ndarray,
    multi_logi_classifier: MultiLogisticReg,
    etas: List[float],
    classifier_kwargs: Dict[str, Any] = {},
    sgd_kwargs: Dict[str, Any] = {},
    verbose: bool = True,
) -> List[MultiLogisticReg]:
    '''
    Returns trained classifiers, instantiated with classifier_kwargs
    and trained with sgd_kwargs. Each trained classifier to the
    corresponding learning rate eta in etas.

    Args:
        xTr: Training raw features. shape (mTr, *)
        yTr: Training labels. shape (mTr)
        xVal: Validation raw features. shape (mVal, *)
        yVal: Validation labels. shape (mVal)
        etas: A list of learning rate values (etas).
        multi_logi_classifier: Class name of the multiclass logistic classifier.
        classifier_kwargs: (default {}) Keyword arguments to instantiate
            multiclass logistic classifier, supplied as a dictionary.
        sgd_kwargs: (default {}) Keyword arguments for SGD training,
            supplied as a dictionary.
        verbose: (default True) Flag for printing progress.

    Returns:
        clfs: List of trained classifiers. clfs[i] classifier is trained
            with eta=etas[i].
    '''
    sgd_kwargs['verbose'] = sgd_kwargs.pop('verbose', True) and verbose
    clfs = []
    for eta in etas:
        if verbose:
            print(f'eta: {eta:.2e} ::: ', end='')

        # Train classifier
        clf = multi_logi_classifier(**classifier_kwargs)
        clf.fit(xTr, yTr, eta=eta, **sgd_kwargs)

        # Log train and val err
        yTr_pred = clf.predict(xTr)
        train_err = utils.empirical_err(yTr, yTr_pred)
        train_err_logger = SGDLogger(
            'train_err', None, train_err, can_display=True, per_epoch=False
        )
        clf.sgd_loggers.append(train_err_logger)

        yVal_pred = clf.predict(xVal)
        val_err = utils.empirical_err(yVal, yVal_pred)
        val_err_logger = SGDLogger(
            'val_err', None, val_err, can_display=True, per_epoch=False
        )
        clf.sgd_loggers.append(val_err_logger)

        if verbose:
            print(f'train_err: {train_err:5f}, val_err: {val_err:5f}')

        clfs.append(clf)

    val_errs = [logger.log
        for clf in clfs
        for logger in clf.sgd_loggers
        if logger.name == 'val_err']

    train_errs = [logger.log
        for clf in clfs
        for logger in clf.sgd_loggers
        if logger.name == 'train_err']

    best_idx = np.argmin(val_errs)
    best_eta = etas[best_idx]
    best_clf = clfs[best_idx]
    best_val_err = val_errs[best_idx]

    return best_clf, best_eta, best_val_err

In [ ]:
# Values below are a recommendation for initial exploration -- you should see if you want to refine these
etas = np.logspace(-4, 0, 5)

classifier_kwargs = {
    'phi': norm_affine_phi,
    'is_affine_phi': True,
}

sgd_kwargs = {
    'batch_size': 16,
    'n_epochs': 20,
    'verbose': False,
}

best_clf, best_eta, best_val_err = tune_learning_rate(
    xTr, yTr, xVal, yVal,
    MultiLogL2Reg,
    etas,
    classifier_kwargs=classifier_kwargs,
    sgd_kwargs=sgd_kwargs,
    verbose=True
)

print(f"Best validation error trained with feature normalization: {best_val_err}")

#### TASK 6 CODE
def null_phi(x):
    return

classifier_kwargs = {
    'phi': null_phi,
    'is_affine_phi': True,
}







#### TASK 6 CODE

print(f"Best validation error trained without feature normalization: {best_val_err}")

### [Task 7] Comparing Different Sampling Strategies

In the lecture, we introduced mini-batch gradient descent.
In this task, please experiment with the four different sampling strategies for the computation of mini-batch gradients implemented in the `SGD` method.

- `cycling`: cycle over the data in the input order.
- `randperm`: cycle over a random permutation of data fixed across epochs.
- `epoch_shuffle`: cycle over a random permutation of data shuffled randomly every epoch.
- `iid`: get every batch by iid sampling of the data.

Please plot the training error of the classifiers trained with the four sampling strategies as a function of training epochs.
Do you observe significant differences between the different strategies?





In [ ]:
classifier_kwargs = {
    'phi': norm_affine_phi,
    'is_affine_phi': True,
}

sgd_kwargs = {
    'batch_size': 16,
    'eta': 0.1,
    'n_epochs': 20,
    'verbose': False,
}

clfs = []
sampling_methods = ['cyclic', 'randperm', 'epoch_shuffle', 'iid']
#### TASK 7 CODE
def get_train_error(w):
    return

def get_val_error(w):
    return

for sampling_method in sampling_methods:






train_errs =

#### TASK 7 CODE

plt.figure()
for sampling_method, train_err in zip(sampling_methods, train_errs):
    plt.plot(np.arange(sgd_kwargs['n_epochs']), train_err, label=f'{sampling_method}')
plt.ylabel('train err')
plt.xlabel('epochs')
plt.legend(title='sampling method')
plt.show()

### [Task 8] Optimize Learning Rate for Different Batch Sizes

Recall that the formulation of mini-batch gradient descent can be written as:
$$
w_{t+1} \leftarrow w_t - \eta \frac{1}{b} \sum_{i \in B_t} \nabla_w loss(x_i, y_i, w_t)
$$
where $B_t$ represents the mini-batch sampled at time step $t$, and $b = |B_t|$ represents the batch size.
Consider the cases when we train the model for fixed epochs with different batch sizes $b_1, b_2$, where $b_2 \gg b_1$.
Since we have many fewer batches (and iterations) within one epoch when using a large batch size, it is very possible that the model can underfit if we use a fixed learning rate $\eta$.
Therefore, a simple solution is to scale up the learning rate according to the batch size by:
$$
\eta_b = \eta \cdot b
$$


In this task, please experiment with different batch sizes and use the tune_learning_rate method provided in Task 6 to find the optimal learning rate $\eta_b^*$ for different batch sizes.
You should find that the optimal learning rates roughly follow the relation $\frac{\eta_{b_1}^*}{b_1} \approx \frac{\eta_{b_2}^*}{b_2}$.


In [ ]:
classifier_kwargs = {
    'phi': norm_affine_phi,
    'is_affine_phi': True,
}

sgd_kwargs = {
    'n_epochs': 10,
    'verbose': False,
}

batch_sizes = [4, 16, 64, 256]
best_etas = []  # List to store best eta for each batch size

for batch_size in batch_sizes:
    #### TASK 8 CODE
    etas = np.logspace(-4, 0, 8)




    # Store best_eta
    #### TASK 8 CODE
    print(f"Batch size = {batch_size}, Best learning rate = {best_eta}")

### [Task 9] Improving Training Efficiency by Increasing Batch Sizes

The benefits of gradient descent with large batch sizes include (1) more accurate estimation of gradients and (2) improved efficiency via parallel computation.
In this task, we aim to compare the training efficiency of models trained with different batch sizes.

In the first part, we use a fixed learning rate across different batch sizes.
This setup, however, is not ideal, as we just discussed in Task 8.


In [ ]:
# With fixed learning rates
classifier_kwargs = {
    'phi': norm_affine_phi,
    'is_affine_phi': True,
}

sgd_kwargs = {
    'eta': 0.1,
    'n_epochs': 20,
    'verbose': False,
}

import time
def log_time(w):
    return time.time()

clfs = []
batch_sizes = [4, 16, 64, 256, 512, 1024]
for batch_size in batch_sizes:
    sgd_kwargs['batch_size'] = batch_size
    clf = MultiLogL2Reg(**classifier_kwargs)
    sgd_kwargs['loggers'] = [
        SGDLogger('time', log_time, None, can_display=True, per_epoch=True),
    ]

    clf.fit(xTr, yTr, **sgd_kwargs)
    clfs.append(clf)

train_objs = [logger.log
            for clf in clfs
            for logger in clf.sgd_loggers
            if logger.name == 'train_obj']


To analyze the time consumption of training, we can use $batch\_size \times \#\ total\_batches$ as an indicator of the running time for serial computation and use $\#\ total\_batches$ to show the running time for parallel computation.
Practically, the execution of a Python program falls somewhere between the fully-serial and fully-parallel cases, so we encourage you to also compare the actual time consumption of your Python program.

Please plot the training losses with different batch sizes as a function of (1) $batch\_size \times \#\ total\_batches$, (2) $\#\ total\_batches$, and (3) execution time. In the figures, you should obviously see that the models with large batch sizes are still underfitting, as we argued in Task 8.

In [ ]:
plt.figure()
for batch_size, train_obj in zip(batch_sizes, train_objs):
    #### TASK 9 CODE

    #### TASK 9 CODE
plt.ylabel('train loss')
plt.xlabel('parallel runtime (t)')
plt.legend()
plt.show()

### Plotting parallel runtime where x axis stops for all batch sizes at the same value ###
plt.figure()
max_batch_size = max(batch_sizes)
max_x_stop = (sgd_kwargs['n_epochs']) * (len(X) // max_batch_size)
for batch_size, train_obj in zip(batch_sizes, train_objs):
    #### TASK 9 CODE

    #### TASK 9 CODE
plt.ylabel('train loss')
plt.xlabel('parallel runtime (t)')
plt.legend()
plt.show()

plt.figure()
for batch_size, train_obj in zip(batch_sizes, train_objs):
    #### TASK 9 CODE

    #### TASK 9 CODE
plt.ylabel('train loss')
plt.xlabel('serial runtime  (t * b)')
plt.legend()
plt.show()

times = [logger.log
         for clf in clfs
         for logger in clf.sgd_loggers
         if logger.name == 'time']
plt.figure()
for batch_size, train_obj, time in zip(batch_sizes, train_objs, times):
    plt.plot(np.array(time)-time[0], train_obj, label=f'batch size {batch_size}')
plt.ylabel('train loss')
plt.xlabel('python runtime')
plt.legend()
plt.show()

In the second part, we'll follow the strategy outlined in Task 8 to adjust learning rates according to batch sizes.
Repeat what we did in the first part.
In the three plots, we should be able to observe that using a large batch size can achieve similar performance with much less time in the case of parallel computing.


While you should actually be using the code from Task 8 to tune the step-size for each batch size, this may be time consuming. We will use the observation that the optimal learing rates satisify the relation $\frac{\eta_{b_1}^*}{b_1} \approx \frac{\eta_{b_2}^*}{b_2}$.

So for simplicity, just use $\eta_{b}^*=0.025 \times b$ and see what you observe.


In [ ]:
import time
sgd_kwargs = {
    'n_epochs': 20,
    'verbose': False,
}

clfs = []
#### TASK 9 CODE
# Train the models with adjusted learning rates
for batch_size in batch_sizes:
    sgd_kwargs['batch_size'] =
    sgd_kwargs['eta'] =




#### TASK 9 CODE

train_objs = [logger.log
            for clf in clfs
            for logger in clf.sgd_loggers
            if logger.name == 'train_obj']

In [ ]:
plt.figure()
for batch_size, train_obj in zip(batch_sizes, train_objs):
    #### TASK 9 CODE

    #### TASK 9 CODE
plt.ylabel('train loss')
plt.xlabel('parallel runtime (t)')
plt.legend()
plt.show()

plt.figure()
for batch_size, train_obj in zip(batch_sizes, train_objs):
    #### TASK 9 CODE

    #### TASK 9 CODE
plt.ylabel('train loss')
plt.xlabel('serial runtime  (t * b)')
plt.legend()
plt.show()

times = [logger.log
         for clf in clfs
         for logger in clf.sgd_loggers
         if logger.name == 'time']
plt.figure()
for batch_size, train_obj, time in zip(batch_sizes, train_objs, times):
    plt.plot(np.array(time)-time[0], train_obj, label=f'batch size {batch_size}')
plt.ylabel('train loss')
plt.xlabel('python runtime')
plt.legend()
plt.show()

 **What do you observe when tuning the the step size for batch sizes (we considered so far)? What happens to parallel runtime and serial runtime to achieve a fixed desired loss?**

Answer:

**We now ask would the same trend continue if we further increase the batch size (even if we tune the step sized appropriately)? Let's explore this!**

In [ ]:
batch_sizes = [2048, 4096, 8192]

Let's exactly find the best step sizes for these batch sizes using the code from Task 8.

In [ ]:
classifier_kwargs = {
    'phi': norm_affine_phi,
    'is_affine_phi': True,
}

sgd_kwargs = {
    'n_epochs': 100,
    'verbose': False,
}
best_etas = []  # List to store best eta for each batch size

for batch_size in batch_sizes:
    #### TASK 9 CODE
    etas = np.logspace(-1, 1, 3)
    ##### Copy your code from Task 8 to tune learning rate in this range


    # Store best_eta
    #### TASK 9 CODE
    print(f"Batch size = {batch_size}, Best learning rate = {best_eta}")

In [ ]:
import time
sgd_kwargs = {
    'n_epochs': 100,
    'verbose': False,
}

clfs = []
#### TASK 9 CODE
# Train the models with adjusted learning rates
for (batch_size,best_eta) in zip(batch_sizes,best_etas):
    sgd_kwargs['batch_size'] =
    sgd_kwargs['eta'] =


#### TASK 9 CODE

train_objs = [logger.log
            for clf in clfs
            for logger in clf.sgd_loggers
            if logger.name == 'train_obj']

In [ ]:
plt.figure()
for batch_size, train_obj in zip(batch_sizes, train_objs):
    #### TASK 9 CODE

    #### TASK 9 CODE
plt.ylabel('train loss')
plt.xlabel('parallel runtime (t)')
plt.legend()
plt.show()


plt.figure()
for batch_size, train_obj in zip(batch_sizes, train_objs):
    #### TASK 9 CODE

    #### TASK 9 CODE
plt.ylabel('train loss')
plt.xlabel('serial runtime  (t * b)')
plt.legend()
plt.show()

times = [logger.log
         for clf in clfs
         for logger in clf.sgd_loggers
         if logger.name == 'time']
plt.figure()
for batch_size, train_obj, time in zip(batch_sizes, train_objs, times):
    plt.plot(np.array(time)-time[0], train_obj, label=f'batch size {batch_size}')
plt.ylabel('train loss')
plt.xlabel('python runtime')
plt.legend()
plt.show()

**What do you observe? What happened to the trend when we further increased the batch size?**

Answer: As we increas the batch size further, the parallel runtime no longer improved, but the serial runtime increased linearly.

### [Task 10] Final classifier

As the last task, train a classifier with hyperparameters of your choice.
Plot the training/validation loss & error as a function of training iterations, and report the final empirical errors on training and validation dataset splits.

In [ ]:
#### TASK 10 CODE
def get_val_obj(w):
    return np.mean(clf.loss_func(norm_affine_phi(xVal) @ w, yVal))




#### TASK 10 CODE

train_err = utils.empirical_err(yTr, best_clf.predict(xTr))
val_err = utils.empirical_err(yVal, best_clf.predict(xVal))


print(f'Train err: {train_err:0.5f}')
print(f'Validation err: {val_err:0.5f}')

### [Task 11] Test Result and Kaggle

We are now ready to use our shipped predictor on the test examples `xTe`! You are required to submit predictions on the [Kaggle competition ](https://www.kaggle.com/t/bd133eebbc074c7e9e93e7e965676d3d)as a csv file, using the provided code below. And then you must report your accuracy score from Kaggle, which should match with the benchmark! This part is mendatory.

In [ ]:
#### Task 11 Code ####
test_predictions = best_clf.predict(xTe)

In [ ]:
print(f"My predictions: {test_predictions}") ### this should be a list with entries {0,..,.9} ###
print(f"It if of length {len(test_predictions)}.") ### this should of size X_test = 10000 ###

Now let's export it as a CSV file and make a Kaggle submission.

In [ ]:
with open("y_predicted.csv", "w") as f:
    f.write("ID,label\n")  # Write header
    for i, label in enumerate(test_predictions, start=1):
        f.write(f"{i},{label}\n")  # Write ID and label

<span style="color: blue">

Answer:


My test Accuracy is =
</span>

### [Task 12] [Challenge] SGD with Different Feature Maps

As a challenge, we encourage you to implement SGD with potentially different reature maps, than the naive one we used above. You may also implement the kernalized SGD that we saw in Problem 1 Theory and try out different kernels. Please reuse the code above to generate test prediction on `xTe`, convert into a csv file, and submit on Kaggle to compete with fellow students! And, more importantly, also have fun along the way!